## Imports and Base models

In [55]:
from typing import Literal
from langchain import hub
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
import os
load_dotenv()

True

In [56]:
gemini_token = os.getenv("GEMINI_TOKEN")
structurated_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_token)

In [57]:
hf_token = os.getenv("HF_TOKEN")
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3.1-70B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=hf_token
)
chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Query Analysis

In [58]:
# Data Model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource"""

    datasource: Literal["vectorstore", "duckduckgo", "github", 'notify', 'none'] = Field(
        ...,
        description="Given a user question choose to route to the most relevant datasource"
    )

LLM_router = structurated_model.with_structured_output(RouteQuery)

In [62]:
# Prompt
system = """You are PampuAI, an expert at routing a user question to different tools.
The 'vectorstore' contains documents, including martin's cv and all actions that you can perform (PampuAI).
Use 'duckduckgo' for questions that can be answered by a web search.
Use 'github' for questions about martin's recent projects.
Use 'notify' to send a notification with a message to martin.
Use 'none' for chatting with PampuAI.
"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | LLM_router

In [61]:
print(question_router.invoke({"question": "Que tecnologias maneja martin?"}))
print(question_router.invoke({"question": "Cual es el nombre de Obama?"}))
print(question_router.invoke({"question": "En que proyectos trabajo martin ultimamente?"}))
print(question_router.invoke({"question": "Decile a martin que me gusto su portfolio"}))
print(question_router.invoke({"question": "Que podes hacer?"}))
print(question_router.invoke({"question": "Hola!"}))

datasource='vectorstore'
datasource='duckduckgo'
datasource='github'
datasource='notify'
datasource='vectorstore'
datasource='none'


# Tools

## DuckDuckGo

# Implementation

In [60]:
while True:
    question = input("You: ")
    if question == "exit":
        break

    # Route the question
    response = question_router.invoke({"question": question})
    print("Selected tool:", response.datasource)

    if response.datasource == "notify":
        print("Sending notification...")

    elif response.datasource == "none":
        print("Chatting with PampuAI...")

    elif response.datasource == "vectorstore":
        print("Searching in vectorstore...")

    elif response.datasource == "duckduckgo":
        print("Searching in duckduckgo...")

    elif response.datasource == "github":
        print("Searching in github...")

Selected tool: none
Chatting with PampuAI...
Selected tool: duckduckgo
Searching in duckduckgo...
Selected tool: vectorstore
Searching in vectorstore...
Selected tool: notify
Sending notification...
